In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import timedelta

def combine_csv_files(directory):
    """
    This function combines multiple CSV files from a given directory into a single pandas DataFrame. It also performs some data cleaning and transformation tasks.

    Parameters:
    directory (str): The path to the directory containing the CSV files.

    Returns:
    combined_df (pd.DataFrame): A DataFrame containing the combined data from all CSV files. The DataFrame includes the following columns:
        - 'datetime': Timestamps converted to pandas datetime objects.
        - 'dayofweek', 'objectname', 'datatype', 'avgvalue', 'hourending', 'marketday', 'year', 'month': Columns directly taken from the original CSV files.
        - 'peaktype': This column is included if it exists in the original CSV files.
        - 'datatype': This column's values are replaced based on a predefined mapping.

    Note:
    The function assumes that all CSV files have a similar structure and column names.
    """

    dfs = []

    for filename in os.listdir(directory):
        
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        df['datetime'] = pd.to_datetime(df['datetime'])
        dfs.append(df)

    combined_df = pd.concat(dfs, ignore_index=True)
    if 'peaktype' in combined_df.columns:
        combined_df = combined_df[['datetime', 'dayofweek', 'objectname', 'datatype', 'avgvalue', 'hourending', 'marketday', 'peaktype', 'year', 'month']]
        combined_df['datatype'] = combined_df['datatype'].replace({'RTLOAD': 'Load', 'GENERATION_SOLAR_RT': 'Solar', 'WINDDATA': 'Wind', 'TOTAL_RESOURCE_CAP_OUT': 'Outage'})
    else:
        combined_df = combined_df[['datetime', 'dayofweek', 'objectname', 'datatype', 'avgvalue', 'hourending', 'marketday', 'month', 'year']]
        combined_df['datatype'] = combined_df['datatype'].replace({'LOAD_FORECAST': 'Load', 'SOLAR_STPPF': 'Solar', 'WIND_STWPF': 'Wind', 'TOTAL_RESOURCE_CAP_OUT': 'Outage'})
    return combined_df

def datetime_split(df):
    """
    Splits the 'datetime' column of a DataFrame into separate columns for year, month, day, hour, and week.

    Parameters:
    df (pandas.DataFrame): Input DataFrame with a 'datetime' column in string format.

    Returns:
    df (pandas.DataFrame): Output DataFrame with the 'datetime' column replaced by 'year', 'month', 'day', 'hour', and 'week' columns. The 'datetime' column is converted to datetime format, and the new columns are derived from it.

    The 'year', 'month', and 'day' columns are self-explanatory. The 'hour' column contains the time information. The 'week' column contains the ISO week number.

    Example:
    >>> df = pd.DataFrame({'datetime': ['2022-01-01 13:45:30']})
    >>> datetime_split(df)
       year  month  day      hour  week
    0  2022      1    1  13:45:30     1
    """
   
    df['datetime'] = pd.to_datetime(df['datetime'])

    df['year'] = df['datetime'].dt.year
    df['month'] = df['datetime'].dt.month
    df['day'] = df['datetime'].dt.day
    df['hour'] = df['datetime'].dt.time
    df['week'] = df['datetime'].dt.isocalendar().week

    return df

def Merge(Actual, Forecast):

    """
    Merges two dataframes containing actual and forecast data on shared columns, computes residuals, and prepares a comprehensive dataframe.

    This function merges the 'Actual' and 'Forecast' dataframes based on common columns: 'datetime', 'datatype', and 'objectname'. It performs an inner join to ensure that only rows present in both dataframes are included in the merged dataframe. The function then calculates the residuals (difference between actual and forecasted values), generates a 'Region_type' column by concatenating 'objectname' and 'datatype', and performs further data manipulation to structure the dataframe as required. It also splits the 'datetime' column into additional time-related columns and selects a specific set of columns for the final output.

    Parameters:
    - Actual (pandas.DataFrame): The dataframe containing actual data with columns including 'datetime', 'datatype', 'objectname', and others relevant to actual measurements.
    - Forecast (pandas.DataFrame): The dataframe containing forecast data with columns matching those of the Actual dataframe necessary for merging and comparison.

    Returns:
    - pandas.DataFrame: A merged dataframe with columns for datetime, day of the week, object name, data type, actual measurements, hour ending, forecast measurements, peak type, residuals, region type, and split datetime components (year, month, day, hour).

    Notes:
    - The function assumes that 'datetime_split' is a predefined function that splits the 'datetime' column into 'year', 'month', 'day', and 'hour' columns.
    - Columns 'year' and 'month' are dropped after the merge to streamline the dataframe before applying 'datetime_split'.
    - The 'dayofweek', 'hour ending', and 'peaktype' columns are assumed to be part of the Actual dataframe and are carried over into the merged dataframe.
    """

    Merged = Actual.merge(Forecast, how = 'inner', left_on = ['datetime', 'datatype', 'objectname'], right_on = ['datetime', 'datatype', 'objectname'])
    Merged = Merged.rename(columns = {'datetime_x': 'datetime', 'dayofweek_x': 'dayofweek', 'avgvalue_x': 'Actual MW', 'year_x': 'year', 'month_x': 'month', 'hourending_x': 'hour ending', 'avgvalue_y': 'Forecast MW'})
    Merged['residuals'] = Merged['Actual MW'] - Merged['Forecast MW']
    Merged['Region_type'] = Merged[['objectname', 'datatype']].apply(lambda row: ' '.join(map(str, row)), axis=1)
    Merged = Merged.drop(['year', 'month'], axis = 1)
    Merged = datetime_split(Merged)
    Merged = Merged[['datetime', 'dayofweek', 'objectname', 'datatype', 'Actual MW', 'hour ending', 'Forecast MW','peaktype', 'residuals', 'Region_type', 'year', 'month', 'day', 'hour']]
    return Merged


def Combined_Files(Actual_directory, Forecast_directory):
    """
    This function merges two dataframes, Actual and Forecast, based on certain common columns. It then performs a series of transformations and computations on the merged dataframe.

    Parameters:
    Actual (DataFrame): The actual data, expected to have columns ['datetime', 'datatype', 'objectname'].
    Forecast (DataFrame): The forecasted data, expected to have columns ['day_ahead', 'datatype', 'objectname'].

    Returns:
    Merged (DataFrame): The merged dataframe with columns ['datetime', 'dayofweek', 'objectname', 'datatype', 'Actual MW', 'hour ending', 'Forecast MW','peaktype', 'residuals', 'Region_type', 'year', 'month', 'day', 'hour'].

    The returned dataframe includes calculated residuals (the difference between actual and forecasted values), a combined region type, and split datetime into year, month, day, and hour.
    """
    
    Actuals = combine_csv_files(Actual_directory)
    Forecasts = combine_csv_files(Forecast_directory)

    Merged = Merge(Actuals, Forecasts)

    return Merged


def percentile_dataframe(Actual_directory, Forecast_directory, Output_directory, Output_file_name):

    """
    Generates a dataframe of percentile values for residuals, grouping by object name, data type, year, month, and day.

    This function performs the following operations:
    - Merges actual and forecast data from their respective directories into a single DataFrame.
    - Groups the merged data by object name, data type, year, month, and day, and calculates the 10th, 25th, 50th, 75th, and 90th percentiles of residuals.
    - Renames the columns for clarity and modifies the percentile column to include descriptive labels (e.g., "10th" for 0.1).
    - Converts the month, year, and day columns to strings, and then combines them to form a new 'date' column in 'MM/DD/YYYY' format.
    - Converts the new 'date' column to datetime format.
    - Saves the resultant DataFrame as a CSV file in the specified output directory with the given output file name. If the output file name does not end with '.csv', the extension is appended automatically.

    Parameters:
    - Actual_directory (str): The directory path where the actual data files are located.
    - Forecast_directory (str): The directory path where the forecast data files are located.
    - Output_directory (str): The directory path where the output CSV file should be saved.
    - Output_file_name (str): The name of the output file. If the name does not end with '.csv', the extension will be added automatically.

    Returns:
    - None: Outputs a message confirming the successful creation of the CSV file.
    - Exports the df to csv

    """
    
    df = Combined_Files(Actual_directory, Forecast_directory)

    perc_df_grouped = df.groupby(['objectname', 'datatype', 'year', 'month', 'day'])['residuals'].quantile([.1,.25,.5,.75,.9]).reset_index()
        
    perc_df_grouped = perc_df_grouped.rename(columns = {'level_5': 'percentile', 'residuals': 'perc_value'})
        
    perc_df_grouped['percentile'] = perc_df_grouped['percentile'].astype(str)
        
    perc_df_grouped['percentile'] = perc_df_grouped['percentile'].replace({'0.1':'10th', '0.25':'25th', '0.5':'50th', '0.75':'75th', '0.9':'90th'})
        
    perc_df_grouped['month'] = perc_df_grouped['month'].astype(str)
        
    perc_df_grouped['year'] = perc_df_grouped['year'].astype(str)
        
    perc_df_grouped['day'] = perc_df_grouped['day'].astype(str)
    
    perc_df_grouped['date'] = perc_df_grouped['month'] + '/' + perc_df_grouped['day'] + '/' + perc_df_grouped['year']
        
    perc_df_grouped['date'] = pd.to_datetime(perc_df_grouped['date'])

    df = df[['objectname', 'datatype', 'year', 'month', 'day', 'dayofweek']]

    df['year'] = df['year'].astype(str)

    df['month'] = df['month'].astype(str)

    df['day'] = df['day'].astype(str)

    perc_df_grouped = perc_df_grouped.merge(df, how = 'left', on = ['objectname', 'datatype', 'year', 'month', 'day'])

    perc_df_grouped = perc_df_grouped.drop_duplicates(subset=['objectname', 'datatype', 'year', 'month', 'day', 'percentile'])
    
    if '.csv' in Output_file_name:
        path = Output_directory + Output_file_name
    else:
        path = Output_directory + Output_file_name + '.csv'

    perc_df_grouped.to_csv(path)

    return print("Succesfully created the csv!")

    
def residual_dataframe(Actual_directory, Forecast_directory, Output_directory, Output_file_name):

    """
    Creates a residual dataframe by merging actual and forecast data, then saves it as a CSV file.

    This function merges two data sets located in specified directories: one containing actual data and the other containing forecast data. The merged data set is then saved as a CSV file in the specified output directory with the given output file name. If the output file name does not end with '.csv', the extension is appended automatically.

    Parameters:
    - Actual_directory (str): The directory path where the actual data files are located.
    - Forecast_directory (str): The directory path where the forecast data files are located.
    - Output_directory (str): The directory path where the output CSV file should be saved.
    - Output_file_name (str): The name of the output file. If the name does not end with '.csv', the extension will be added automatically.

    Returns:
    - None: Outputs a message confirming the successful creation of the CSV file.
    - Exports the df to csv
    """

    Merged = Combined_Files(Actual_directory, Forecast_directory)

    if '.csv' in Output_file_name:
        path = Output_directory + Output_file_name
    else:
        path = Output_directory + Output_file_name + '.csv'

    Merged.to_csv(path)

    return print("Succesfully created the csv!")
    
def price_dataframe(Actual_directory, Forecast_directory, Price_directory, Output_directory, Output_file_name):

    """
    Merges actual, forecast, and price data from specified directories, then saves the merged dataframe as a CSV file.

    This function performs the following operations:
    - Reads price data from a directory and combines these files into a single DataFrame.
    - Merges actual and forecast data from their respective directories into a single DataFrame.
    - Replaces certain object names in the price data with predefined values.
    - Merges the price DataFrame with the combined actual and forecast data DataFrame based on datetime and object name.
    - Selects and renames specific columns in the merged DataFrame.
    - Removes duplicate rows based on datetime, object name, and data type.
    - Converts the price column to float type.
    - Saves the final DataFrame as a CSV file in the specified output directory with the given output file name. If the output file name does not end with '.csv', the extension is appended automatically.

    Parameters:
    - Actual_directory (str): The directory path where the actual data files are located.
    - Forecast_directory (str): The directory path where the forecast data files are located.
    - Price_directory (str): The directory path where the price data files are located.
    - Output_directory (str): The directory path where the output CSV file should be saved.
    - Output_file_name (str): The name of the output file. If the name does not end with '.csv', the extension will be added automatically.

    Returns:
    - None: Outputs a message confirming the successful creation of the CSV file.
    - Exports the df to csv
    
    """

    Price_df = combine_csv_files(Price_directory)

    df = Combined_Files(Actual_directory, Forecast_directory)
        
    Price_df['objectname'] = Price_df['objectname'].replace({'LZ_WEST': 'WEST (ERCOT)', 'LZ_SOUTH': 'SOUTH', 'LZ_HOUSTON': 'HOUSTON', 'LZ_NORTH': 'NORTH'})
        
    df['datetime'] = pd.to_datetime(df['datetime'])
        
    Combined = df.merge(Price_df, how ='left', on = ['datetime', 'objectname'])

    Combined = Combined[['datetime', 'dayofweek_x', 'objectname', 'datatype_x','Actual MW', 'hour ending', 'peaktype_x',  'Forecast MW', 'residuals', 'Region_type', 'year_x', 'month_x', 'day', 'hour', 'avgvalue']]

    Combined = Combined.rename(columns={'dayofweek_x': 'day of week', 'datatype_x': 'datatype', 'peaktype_x': 'peaktype', 'year_x': 'year', 'month_x': 'month', 'avgvalue': 'price'})

    Combined.drop_duplicates(subset=['datetime', 'objectname', 'datatype'], keep='first', inplace=True)

    Combined['price'] = Combined['price'].astype(float)

    if '.csv' in Output_file_name:
        path = Output_directory + Output_file_name
    else:
        path = Output_directory + Output_file_name + '.csv'

    Combined.to_csv(path)

    return print("Succesfully created the csv!")


def netload_dataframe(Actual_directory, Forecast_directory, Output_directory, Output_file_name):

        """
        Generates a dataframe with net load calculations based on actual and forecast data, then saves it as a CSV file.

        This function performs the following operations:
        - Merges actual and forecast data from their respective directories into a single DataFrame.
        - Converts the 'datetime' column to datetime format.
        - Pivots the DataFrame to create a multi-level column structure with 'datatype' as the columns and 'Forecast MW', 'Actual MW', 'residuals' as the values.
        - Flattens the multi-level columns and renames them by combining the original column names.
        - Calculates the 'Actual Net Load' by subtracting the sum of actual solar and wind MW from the actual load MW.
        - Calculates the 'Forecast Net Load' by subtracting the sum of forecast solar and wind MW from the forecast load MW.
        - Renames the columns for clarity and removes duplicate rows based on 'datetime' and 'objectname'.
        - Ensures that 'Actual Net Load', 'Actual MW Outage', and 'Forecast Net Load' are of float data type.
        - Saves the final DataFrame as a CSV file in the specified output directory with the given output file name. If the output file name does not end with '.csv', the extension is appended automatically.

        Parameters:
        - Actual_directory (str): The directory path where the actual data files are located.
        - Forecast_directory (str): The directory path where the forecast data files are located.
        - Output_directory (str): The directory path where the output CSV file should be saved.
        - Output_file_name (str): The name of the output file. If the name does not end with '.csv', the extension will be added automatically.

        Returns:
        - None: Outputs a message confirming the successful creation of the CSV file.
        - Exports the created df to csv

        """

        df = Combined_Files(Actual_directory, Forecast_directory)

        df['datetime'] = pd.to_datetime(df['datetime'])

        df = df.pivot_table(index=['datetime', 'objectname'], columns='datatype', values=['Forecast MW', 'Actual MW', 'residuals']).reset_index()

        df.columns = ['{}_{}'.format(col[0], col[1]) for col in df.columns]

        df['Actual Net Load'] = df['Actual MW_Load'] - (df['Actual MW_Solar'] + df['Actual MW_Wind'])

        df['Forecast Net Load'] = df['Forecast MW_Load'] - (df['Forecast MW_Solar'] + df['Forecast MW_Wind'])

        df = df.rename(columns = {'datetime_': 'datetime', 'objectname_': 'objectname'})

        df.drop_duplicates(subset=['datetime', 'objectname'], keep='first', inplace=True)

        df['Actual Net Load'] = df['Actual Net Load'].astype(float)

        df['Actual MW_Outage'] = df['Actual MW_Outage'].astype(float)

        df['Forecast Net Load'] = df['Forecast Net Load'].astype(float)

        if '.csv' in Output_file_name:
            path = Output_directory + Output_file_name
        else:
            path = Output_directory + Output_file_name + '.csv'

        df.to_csv(path)

        return print("Succesfully created the csv!")

def SD_dataframe(Actual_directory, Forecast_directory, Output_directory, Output_file_name):

        """
        Creates a CSV file containing the standard deviation (SD) of residuals for each day, grouped by object name, data type, year, month, and day.

        This function calculates the daily standard deviation of residuals after merging actual and forecast data from their specified directories. The standard deviation is computed for each group defined by object name, data type, year, month, and day. The results are then saved to a CSV file in the specified output directory with the given output file name. If the output file name does not end with '.csv', the extension is appended automatically.

        Parameters:
        - Actual_directory (str): The directory path where the actual data files are located.
        - Forecast_directory (str): The directory path where the forecast data files are located.
        - Output_directory (str): The directory path where the output CSV file should be saved.
        - Output_file_name (str): The name of the output file. If the name does not end with '.csv', the extension will be added automatically.

        Returns:
        - None: Outputs a message confirming the successful creation of the CSV file.
        - Exports the created df to csv

        """
    
        df = Combined_Files(Actual_directory, Forecast_directory)

        df_sd = df.groupby(['objectname', 'datatype', 'year', 'month', 'day'])['residuals'].std().reset_index()

        df_sd.rename(columns = {'residuals': 'Daily_SD'}, inplace = True)

        if '.csv' in Output_file_name:
            path = Output_directory + Output_file_name
        else:
            path = Output_directory + Output_file_name + '.csv'

        df.to_csv(path)
        return print("Succesfully created the csv!")

percentile_dataframe('/work/ActualData/', '/work/ForecastData/', '/work/FINAL DATAFRAMES/', 'Percentile1')


Succesfully created the csv!


In [ ]:
percentile_dataframe('/work/ActualData/', '/work/ForecastData/', '/work/FINAL DATAFRAMES/', 'Percentile1')
SD_dataframe('/work/ActualData/', '/work/ForecastData/', '/work/FINAL DATAFRAMES/', 'Standard_deviation1')
netload_dataframe('/work/ActualData/', '/work/ForecastData/', '/work/FINAL DATAFRAMES/', 'Netload1')
price_dataframe('/work/ActualData/', '/work/ForecastData/', '/work/Prices_02_27_24/Prices RTLMP/', '/work/FINAL DATAFRAMES/', 'Price1')
residual_dataframe('/work/ActualData/', '/work/ForecastData/', '/work/FINAL DATAFRAMES/', 'Residual1')

Succesfully created the csv!


# 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=65a70a80-d65f-4818-bf93-0169941c47a8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>